In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, GlobalAvgPool2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow.keras.applications.vgg16
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from keras.utils import to_categorical

In [4]:
# get URLs for testing and training 
train_link = "/kaggle/input/fer2013/train"
test_link = "/kaggle/input/fer2013/test"

## Augment the dataset

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   featurewise_center=False,
                                   featurewise_std_normalization=False,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

test_datagen  = ImageDataGenerator(rescale = 1./255)

In [6]:
train_dataset  = train_datagen.flow_from_directory(directory = train_link,
                                                   target_size = (48,48),
                                                   class_mode = 'categorical',
                                                   batch_size = 32)

test_dataset = test_datagen.flow_from_directory(directory = test_link,
                                                  target_size = (48,48),
                                                  class_mode = 'categorical',
                                                  shuffle = True,
                                                  batch_size = 32)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


## Import VGG16 with imagenet weights

In [8]:
base_model = tf.keras.applications.VGG16(input_shape= (48,48,3),include_top=False,weights="imagenet")
base_model.summary()

58889256/58889256 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                              

## Freeze 12 layers to use pretrained weights

In [10]:
for layer in base_model.layers[:11]:
    layer.trainable=False

## Create model using Tensorflow API

In [12]:
i = Input(shape=(48,48,3))
x = base_model(i)
x = GlobalAvgPool2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(64, activation="relu")(x)
x = Dense(7, activation="sigmoid")(x)

model = Model(i, x)

In [13]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d_1  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                           

## Set the optimizers, loss functions and callbacks

In [15]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [16]:
model.compile(optimizer=adam,
             loss="categorical_crossentropy",
             metrics=["accuracy"])

In [17]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 3,verbose = 1,factor = 0.50, min_lr = 1e-10)

In [18]:
# stops training if validation accuracy does not improve 3 epochs in a row
train_stop_callback = EarlyStopping(monitor="val_accuracy", patience=3)

In [ ]:
model.fit(train_dataset, validation_data=test_dataset, epochs=50, callbacks=[train_stop_callback, lrd])